In [1]:
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)
from datetime import datetime
import numpy as np

In [2]:
data = pd.read_csv('01_data.csv', sep=';', parse_dates=['fulldate'])
etalon = pd.read_csv('02_etalon.csv', sep=';')

In [3]:
#Сгруппируем данные по CID и hash_id и посчитаем количество значений в группе
data2 = data[['cid', 'hash_id']].groupby(['cid', 'hash_id'], as_index=False).size().sort_values(by='hash_id', ascending=False)
data2.head(3)

,cid,hash_id,size
325511,12822,2999841,2
1233955,197981206,2999841,1
1616783,231250434,2999841,1


In [4]:
#Сделаем сводную таблицу по каждому клиенту (hash_id), значения заполним - количеством значений в группе 
data2 = data2.pivot(index='hash_id', columns='cid', values = 'size').fillna(0).reset_index()
data2.head(3)

cid,hash_id,1,2,3,4,5,6,7,8,9,...,254850304,254850305,254850306,254850307,254850308,254850309,254855680,254855681,254855683,254855684
0,1000773,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,1000808,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,1000868,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [5]:
#удалим колонку с идентификатором клиента, чтобы потом сделать вектор по каждому клиенту
data3 = data2.drop('hash_id', axis=1)
data3.head(3)

cid,1,2,3,4,5,6,7,8,9,10,...,254850304,254850305,254850306,254850307,254850308,254850309,254855680,254855681,254855683,254855684
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [6]:
#Сделаем вектор по каждому клиенту
vector=[]
for idx, row in data3.iterrows():
    p=row.to_numpy()
    vector.append(p)

vector[:3]

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.])]

# Посчитаем косинусное сходство между векторами

In [7]:
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
#Сформируем матрицу косинусного сходства между векторами
vector_sparse = sparse.csr_matrix(vector)
similarities_sparse = cosine_similarity(vector_sparse,dense_output=False)

In [9]:
#Переведем матрицу в pandas dataframe. index1 и index2 = номер строки (индексы) между которыми посчитано сходство (similarity)
res_sim = similarities_sparse.tocoo(copy=False)

result_df = pd.DataFrame({'index1': res_sim.row, 'index2': res_sim.col, 'similarity': res_sim.data}
                 )[['index1', 'index2', 'similarity']].sort_values(['index1', 'index2']
                 ).reset_index(drop=True)
result_df.head(3)


,index1,index2,similarity
0,0,0,1.00
1,0,1,0.00
2,0,2,0.00


In [10]:
#Удалим те строки, где максимальное сходство это один и тот же hash_id (index1 = index2)
result_clean_dub = result_df[result_df['index1'] != result_df['index2']].sort_values(by='similarity', ascending=False)
result_clean_dub.head(3)

,index1,index2,similarity
11593642,3032,2080,1.00
7935809,2080,3032,1.00
6207269,1627,3569,1.00


In [11]:
#Отберем максимальное сходство по каждому клиенту
result_clean_dub2 = result_clean_dub.groupby('index1').head(1)

In [12]:
#Сформируем словарь соответствия index1 == index2
idx_hash_id = dict(zip(result_clean_dub2.index1, result_clean_dub2.index2))

In [13]:
#Сформируем словарь соответствия index строки == hash_id
data2_hash = dict(zip(data2.index, data2.hash_id))

In [14]:
#Добавить hash_id
result_clean_dub2['hash_id_id1'] = result_clean_dub2['index1'].map(data2_hash)
result_clean_dub2['hash_id_id2'] = result_clean_dub2['index2'].map(data2_hash)

result_clean_dub2.head(3)

C:\Users\sselu\AppData\Local\Temp\ipykernel_13780\2529859228.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_clean_dub2['hash_id_id1'] = result_clean_dub2['index1'].map(data2_hash)
C:\Users\sselu\AppData\Local\Temp\ipykernel_13780\2529859228.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_clean_dub2['hash_id_id2'] = result_clean_dub2['index2'].map(data2_hash)


,index1,index2,similarity,hash_id_id1,hash_id_id2
11593642,3032,2080,1.00,2335199,1924661
7935809,2080,3032,1.00,1924661,2335199
6207269,1627,3569,1.00,1713536,2585414


# Проверим качество на эталонных данных

In [15]:
from sklearn.metrics import accuracy_score

In [16]:
etalon.head(3)

,id1,id2
0,1361396,2695335
1,1795864,2458905
2,1543059,2730453


In [17]:
#Сформируем словарь соответствия hash_id_id1 == hash_id_id2
hash_2_hash = dict(zip(result_clean_dub2.hash_id_id1, result_clean_dub2.hash_id_id2))

In [18]:
etalon['new_predict'] = etalon['id1'].map(hash_2_hash)
accuracy = accuracy_score(etalon['id2'], etalon['new_predict'])

In [19]:
print(f'Точность модели составляет {accuracy}')

Точность модели составляет 0.8207739307535642


# Сохраним результат соответствия id1, id2

In [24]:
data_to_save = result_clean_dub2[['hash_id_id1','hash_id_id2']].rename({'hash_id_id1': 'id1', 'hash_id_id2': 'id2'}, axis=1)
data_to_save.head(3)

,id1,id2
11593642,2335199,1924661
7935809,1924661,2335199
6207269,1713536,2585414


In [25]:
data_to_save.to_csv('predict.csv', sep=';', index=False)